![](http://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/UPM/EscPolitecnica/EscUpmPolit_p.gif "UPM")

# Trabajo final SITC
## Análisis de sentimientos en Twitter

Departamento de Ingeniería de Sistemas Telemáticos. Universidad Politécnica de Madrid.

Realizado por:
- Juan Bermudo Mera
- Margarita Bolívar Jiménez
- Lourdes Fernández Nieto
- Ramón Pérez Hernández

© 2017

# Algoritmo GaussianNB aplicado sobre el fichero de tweets

## Tabla de contenidos

* [Importación de datos necesarios para aplicar el algoritmo](#1.-Importación-de-datos-necesarios-para-aplicar-el-algoritmo)
	* [Importación de librerías](#Importación-de-librerías)
    * [Importación de corpus y tweets](#Importación-de-corpus-y-tweets)
    * [Tokenización y stemming](#Tokenización-y-stemming)
* [Entrenamiento del modelo](#2.-Entrenamiento-del-modelo)
* [Rendimiento del modelo](#3.-Rendimiento-del-modelo)
* [Predicción de la polaridad](#4.-Predicción-de-la-polaridad)

## 1. Importación de datos necesarios para aplicar el algoritmo

* ### Importación de librerías

In [1]:
# Importamos librerías. Las que no están instaladas, instalar con pip install <nombre_paquete>
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from mlxtend.preprocessing import DenseTransformer

/Users/Lourdes/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Lourdes/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


* ### Importación de corpus y tweets

In [2]:
# Se importan el corpus y los tweets
tweets_df = pd.read_excel('ficheros/Preprocesados/tweets_corpus_header.xlsx', header=0, encoding='iso8859_15')
tweets = pd.read_excel('ficheros/TweetsConTopic/tweets_spainGeo_topic.xlsx', header=0, encoding='iso8859_15')

* ### Tokenización y stemming

In [3]:
# Se descargan las palabras de parada en español
nltk.download("stopwords")
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Lourdes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Obtenemos los signos de puntuación que se utilizan en español
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))

In [5]:
# Se definen las funciones para realizar la tokenización y el stemming
stemmer = SnowballStemmer('spanish')
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # Eliminamos lo que no sean palabras
    text = ''.join([c for c in text if c not in non_words])
    # Tokenización
    tokens = tknzr.tokenize(text)

    # Stemming
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

## 2. Entrenamiento del modelo

In [6]:
# Buscamos los parámetros que podemos utilizar para entrenar el modelo
GaussianNB().get_params()

{'priors': None}

El algortimo GaussianNB, tiene un parámetro denominado priors. Este parámetro especifica las probabilidades previas de las clases. En nuestro caso, no queremos especificarlas porque queremos que las probabilidades se ajusten según los datos --> No hace falta que busquemos cuáles son los mejores parámetros que se ajusten al modelo --> No realizamos la fase de entrenamiento y pasamos directamente a conocer cuál es el rendimiento del modelo y a realizar la predicción de polaridad de los tweets con este modelo.


## 3. Rendimiento del modelo

In [7]:
# Mediante validación cruzada obtenemos el rendimiento del modelo
model = GaussianNB()
vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = spanish_stopwords,
    min_df = 0,
    max_df = 4700,
    max_features=1000
)

tweets_df_data_features = vectorizer.fit_transform(tweets_df.content)
tweets_df_data_features_nd = tweets_df_data_features.toarray()

scores = cross_val_score(
    model,
    tweets_df_data_features_nd[0:len(tweets_df)],
    y=tweets_df.polarity_bin,
    scoring='roc_auc',
    cv=None
    )

scores.mean()

0.79556041006056721

El valor que se obtiene del rendimiento del modelo para la métrica de Área bajo la Curva ROC es de 0.79556041006056721

## 4. Predicción de la polaridad

In [6]:
# Una vez que tenemos el modelo que mejor métrica nos aporta (tras realizar muchas pruebas con distintas métricas 
# y parámetros pasados al modelo), volvemos a crear un pipeline pero en este caso, pasándole los mejores parámetros
# obtenidos del SVC para predecir qué polaridad tienen los tweets que están aún sin etiquetar
pipeline = Pipeline([
    ('vect', CountVectorizer(
            analyzer = 'word',
            tokenizer = tokenize,
            lowercase = True,
            stop_words = spanish_stopwords,
            min_df = 0,
            max_df = 26363,
            max_features=1000
            )),
    ('to_dense', DenseTransformer()),
    ('cls', GaussianNB())
])

pipeline.fit(tweets_df.content, tweets_df.polarity_bin)
tweets['polarity'] = pipeline.predict(tweets.content)

In [7]:
# Mostramos algunos de los tweets que han sido etiquetados con la polaridad
tweets[['content', 'polarity','Topic']].sample(20)

,content,polarity,Topic
6175,oferta de #empleo para técnico de iluminación ...,0,otros
17211,Te miro y te juro que entre hoja y hoja se te ...,1,otros
1231,¿Tienes una idea más clara de lo que quieres h...,1,otros
568,illo que ya mismo empieza juego de tronossssss...,0,otros
14449,que bonito eso de tenerte ¿no? te quiero @ Alc...,1,otros
8971,Artículos más comentados \n\n…\n\nLeer mas en:...,1,otros
2060,@Angel_Ortiz_G @laSextaTV @_anapastor_ Sigue l...,0,otros
11254,"Cristóbal Rosaleny, @crosaleny es ahora una te...",0,tendencias-twitter
19951,Si te gusta la tortilla de patatas y el pulpo ...,0,otros
13045,Sooy el boss @ Estación de Rivas-Urbanizaciones,0,otros


In [10]:
# Guardamos los tweets en un fichero csv con su polaridad
tweets[[ 'content', 'Latitude', 'Longitude', 'polarity', 'Topic']].to_csv('ficheros/TweetsConPolaridadYTopic/tweetsGaussianNB_polarity_bin.csv', encoding='utf-8')

In [8]:
# Guardamos los tweets en fichero excel
tweets[['content','Latitude','Longitude','polarity','Topic']].to_excel('ficheros/TweetsConPolaridadYTopicExcel/tweetsGaussianNB_polarity_bin.xlsx', header=True, index=False)

<hr>

## Licencia

El notebook está licenciado libremente bajo la licencia [Creative Commons Attribution Share-Alike](https://creativecommons.org/licenses/by/2.0/).

La base del código empleado procede del trabajo de Manuel Garrido llamado [Cómo hacer Análisis de Sentimiento en español](http://pybonacci.org/2015/11/24/como-hacer-analisis-de-sentimiento-en-espanol-2/).

© 2017 - Juan Bermudo Mera, Margarita Bolívar Jiménez, Lourdes Fernández Nieto, Ramón Pérez Hernández.

Universidad Politécnica de Madrid.